# Оценка метрик ReAct Router

Этот ноутбук вычисляет метрики качества работы ReAct Router:
- Recall (Полнота)
- Precision (Точность)
- F1-score
- TP, FP, TN, FN (Confusion Matrix)

ReAct Router решает, нужен ли retriever для ответа на запрос.


In [1]:
import json
import sys
from pathlib import Path
from typing import Literal

import asyncio
import pandas as pd
import numpy as np

# Добавляем путь к проекту
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / "src"))

print(f"📁 Корневая директория проекта: {project_root}")


📁 Корневая директория проекта: /srv/nlp1/T-bank_NLP_-1


In [2]:
from tplexity.generation.generation_service import GenerationService
from tplexity.generation.config import settings

print("✅ Импорты выполнены успешно")


✅ Импорты выполнены успешно


## Загрузка данных


In [3]:
# Путь к датасету
dataset_path = project_root / "data" / "data_18.11.25" / "react_router_dataset.json"

with open(dataset_path, "r", encoding="utf-8") as f:
    dataset = json.load(f)

print(f"📊 Загружено записей: {len(dataset)}")
print(f"\nПример записи:")
print(json.dumps(dataset[0], ensure_ascii=False, indent=2))


📊 Загружено записей: 2896

Пример записи:
{
  "query": "В чем разница между рыночной капитализацией и балансовой стоимостью компании, и как эти показатели используются при оценке акций?",
  "doc_id": "2c8292ce-d169-4f24-9444-a5e32b541b1c",
  "document_text": "Дата публикации: 10 ноября 2025\n\n**🎙**** Это новый выпуск Top News!\n**\nСегодня Ирина Иртегова традиционно расскажет о динамике рынка акций и немного о новостях эмитентов —  **Ozon, МТС Банка, «Ростелекома», «Интер РАО»** и **Группы «Позитив»**. А ещё — [**про результаты ДОМ.PФ**](https://t.me/c/2019406087/30987).\n\n💚 **Еженедельные новости уже ждут вас, инвесторы**!\n\n#OZON #MBNK #RTKM #IRAO #POSI\n\nДата публикации: 10 ноября 2025",
  "expected_router_decision": "NO",
  "requires_rag": false,
  "query_category": "domain_no_rag"
}


## Инициализация GenerationService


In [4]:
# Инициализируем GenerationService
generation_service = GenerationService(
    retriever_url=settings.retriever_api_url,
)

print(f"✅ GenerationService инициализирован")
print(f"   Router LLM Provider: {generation_service.router_llm_provider}")


✅ GenerationService инициализирован
   Router LLM Provider: qwen


## Функция для получения предсказания Router


In [5]:
async def get_router_prediction(query: str, llm_provider: str | None = None) -> Literal["YES", "NO"]:
    """
    Получает предсказание ReAct Router для запроса.
    
    Args:
        query: Запрос пользователя
        llm_provider: Провайдер LLM для принятия решения
    
    Returns:
        "YES" если нужен retriever, "NO" если не нужен
    """
    use_retriever = await generation_service._should_use_retriever(
        query=query,
        session_id=None,  # Без истории диалога
        llm_provider="qwen",
    )
    return "YES" if use_retriever else "NO"


## Оценка на датасете


In [6]:
async def evaluate_dataset(dataset: list[dict], batch_size: int = 10) -> list[dict]:
    """
    Оценивает датасет и возвращает результаты.
    
    Args:
        dataset: Список записей датасета
        batch_size: Размер батча для обработки
    
    Returns:
        Список результатов с предсказаниями
    """
    results = []
    
    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i + batch_size]
        
        # Обрабатываем батч параллельно
        tasks = []
        for record in batch:
            query = record["query"]
            task = get_router_prediction(query)
            tasks.append(task)
        
        predictions = await asyncio.gather(*tasks)
        
        # Сохраняем результаты
        for record, prediction in zip(batch, predictions):
            expected = record.get("expected_router_decision", "YES" if record.get("requires_rag", True) else "NO")
            requires_rag = record.get("requires_rag", expected == "YES")
            
            results.append({
                "query": record["query"],
                "expected": expected,
                "requires_rag": requires_rag,
                "predicted": prediction,
                "correct": prediction == expected,
            })
        
        if (i + batch_size) % 100 == 0 or i + batch_size >= len(dataset):
            print(f"✅ Обработано: {min(i + batch_size, len(dataset))} / {len(dataset)}")
    
    return results


In [7]:
# Запускаем оценку
print(f"🚀 Начинаем оценку на {len(dataset)} запросах...")
results = await evaluate_dataset(dataset, batch_size=50)
print(f"\n✅ Оценка завершена!")


🚀 Начинаем оценку на 2896 запросах...
✅ Обработано: 100 / 2896
✅ Обработано: 200 / 2896
✅ Обработано: 300 / 2896
✅ Обработано: 400 / 2896
✅ Обработано: 500 / 2896
✅ Обработано: 600 / 2896
✅ Обработано: 700 / 2896
✅ Обработано: 800 / 2896
✅ Обработано: 900 / 2896
✅ Обработано: 1000 / 2896
✅ Обработано: 1100 / 2896
✅ Обработано: 1200 / 2896
✅ Обработано: 1300 / 2896
✅ Обработано: 1400 / 2896
✅ Обработано: 1500 / 2896
✅ Обработано: 1600 / 2896
✅ Обработано: 1700 / 2896
✅ Обработано: 1800 / 2896
✅ Обработано: 1900 / 2896
✅ Обработано: 2000 / 2896
✅ Обработано: 2100 / 2896
✅ Обработано: 2200 / 2896
✅ Обработано: 2300 / 2896
✅ Обработано: 2400 / 2896
✅ Обработано: 2500 / 2896
✅ Обработано: 2600 / 2896
✅ Обработано: 2700 / 2896
✅ Обработано: 2800 / 2896
✅ Обработано: 2896 / 2896

✅ Оценка завершена!


## Вычисление метрик


In [11]:
def calculate_metrics(results: list[dict]) -> dict:
    """
    Вычисляет метрики на основе результатов.
    
    Args:
        results: Список результатов с предсказаниями
    
    Returns:
        Словарь с метриками
    """
    # Подсчитываем TP, FP, TN, FN
    # Для ReAct Router:
    # - YES означает "использовать retriever" (positive class)
    # - NO означает "не использовать retriever" (negative class)
    
    tp = 0  # True Positive: predicted=YES, actual=YES (requires_rag=True)
    fp = 0  # False Positive: predicted=YES, actual=NO (requires_rag=False)
    tn = 0  # True Negative: predicted=NO, actual=NO (requires_rag=False)
    fn = 0  # False Negative: predicted=NO, actual=YES (requires_rag=True)
    
    for result in results:
        predicted = result["predicted"]
        requires_rag = result["requires_rag"]
        
        if predicted == "YES" and requires_rag:
            tp += 1
        elif predicted == "YES" and not requires_rag:
            fp += 1
        elif predicted == "NO" and not requires_rag:
            tn += 1
        elif predicted == "NO" and requires_rag:
            fn += 1
    
    # Вычисляем метрики
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    accuracy = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) > 0 else 0.0
    
    return {
        "tp": tp,
        "fp": fp,
        "tn": tn,
        "fn": fn,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": accuracy,
        "total": len(results),
    }


In [12]:
# Вычисляем метрики
metrics = calculate_metrics(results)

print("📊 Результаты оценки ReAct Router:")
print("=" * 60)
print(f"Всего запросов: {metrics['total']}")
print(f"\nConfusion Matrix:")
print(f"  TP (True Positive):  {metrics['tp']:4d}  |  Предсказано YES, фактически YES")
print(f"  FP (False Positive): {metrics['fp']:4d}  |  Предсказано YES, фактически NO")
print(f"  TN (True Negative):  {metrics['tn']:4d}  |  Предсказано NO,  фактически NO")
print(f"  FN (False Negative): {metrics['fn']:4d}  |  Предсказано NO,  фактически YES")
print(f"\nМетрики:")
print(f"  Precision: {metrics['precision']:.4f}  (TP / (TP + FP))")
print(f"  Recall:    {metrics['recall']:.4f}  (TP / (TP + FN))")
print(f"  F1-score:  {metrics['f1']:.4f}")
print(f"  Accuracy:  {metrics['accuracy']:.4f}  ((TP + TN) / Total)")
print("=" * 60)


📊 Результаты оценки ReAct Router:
Всего запросов: 2896

Confusion Matrix:
  TP (True Positive):  1445  |  Предсказано YES, фактически YES
  FP (False Positive):  815  |  Предсказано YES, фактически NO
  TN (True Negative):   633  |  Предсказано NO,  фактически NO
  FN (False Negative):    3  |  Предсказано NO,  фактически YES

Метрики:
  Precision: 0.6394  (TP / (TP + FP))
  Recall:    0.9979  (TP / (TP + FN))
  F1-score:  0.7794
  Accuracy:  0.7175  ((TP + TN) / Total)


In [10]:
# Создаем DataFrame для анализа
df = pd.DataFrame(results)

# Ошибки типа FP (False Positive)
fp_errors = df[(df["predicted"] == "YES") & (df["requires_rag"] == False)]
print(f"\n❌ False Positive (FP) ошибки: {len(fp_errors)}")
print("   Router решил использовать retriever, но это было не нужно")
if len(fp_errors) > 0:
    print("\nПримеры FP ошибок:")
    for idx, row in fp_errors.head(5).iterrows():
        print(f"\n  {idx + 1}. {row['query'][:100]}...")

# Ошибки типа FN (False Negative)
fn_errors = df[(df["predicted"] == "NO") & (df["requires_rag"] == True)]
print(f"\n❌ False Negative (FN) ошибки: {len(fn_errors)}")
print("   Router решил НЕ использовать retriever, но он был нужен")
if len(fn_errors) > 0:
    print("\nПримеры FN ошибок:")
    for idx, row in fn_errors.head(5).iterrows():
        print(f"\n  {idx + 1}. {row['query'][:100]}...")



❌ False Positive (FP) ошибки: 815
   Router решил использовать retriever, но это было не нужно

Примеры FP ошибок:

  1. В чем разница между рыночной капитализацией и балансовой стоимостью компании, и как эти показатели и...

  2. Какие факторы влияют на выбор между долговыми и долевыми инструментами в инвестиционном портфеле?...

  3. Какие факторы влияют на стоимость опционов на акции и как они взаимодействуют между собой?...

  5. Какие стратегии инвестирования можно реализовать с использованием производных инструментов, не требу...

  7. Объясните, почему ожидания изменения ключевой ставки могут влиять на динамику акций в отрасли недвиж...

❌ False Negative (FN) ошибки: 3
   Router решил НЕ использовать retriever, но он был нужен

Примеры FN ошибок:

  1183. Как можно проверить, кто получил доступ к моему аккаунту в Телеграме, и где найти эту информацию?...

  1583. Где можно посмотреть трансляцию пленарной дискуссии конференции «Фокус на клиента»?...

  2158. Какие хэштеги исполь

## Сохранение результатов
